In [19]:
## plot histogram of entropy (grouped by AD diagnosis) for each brain region and frequency.

import glob
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import stats
rng = np.random.default_rng()


output_dir = 'ocd_out05_entropy_histogram/'

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

input_files = glob.glob('ocd_out04_power_histogram_and_entropy/out04_entropy_freq*.csv')
input_files.sort()

def cohend(d1, d2) -> float:

    # calculate the size of samples
    n1, n2 = len(d1), len(d2)

    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)

    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))

    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)

    # return the effect size
    return (u1 - u2) / s


In [20]:
input_files

['ocd_out04_power_histogram_and_entropy/out04_entropy_freq_00.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_01.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_02.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_03.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_04.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_05.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_06.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_07.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_08.csv',
 'ocd_out04_power_histogram_and_entropy/out04_entropy_freq_09.csv']

In [22]:
sns.set(rc = {'figure.figsize':(20,8)})
sns.set_style("whitegrid", {'axes.grid': False})
        
num_roi = 116

## as we selected the frist session as test set (subjects in current files), we use DX_bl as the diagnosis.
## but it seems DX_bl and DX are identical...

column_index = pd.MultiIndex.from_product([[i for i in range(num_roi)], ['p value', 'cohen''s d']])
row_index = np.arange(len(input_files))
result_table = pd.DataFrame(index = row_index, columns = column_index)
result_table.index.name = 'wavelet frequency'    

result_table_emci = result_table.copy()

for f in input_files:
    print(f)
    freq = re.search('(.*)_freq_(.*).csv', f).group(2)
    data = pd.read_csv(f, index_col = 0)
    
    for roi in range(num_roi):
        
        # plot histogram of entropy grouped by DX_bl (AD diagnosis at baseline).
        entropy = data[['group', str(roi)]]
        #entropy = entropy[entropy['DX_bl'].isin(['CN', 'AD', 'EMCI'])]
        n_sample = entropy.groupby('group').count()[str(roi)].min()
        entropy = entropy.groupby("group").sample(n=n_sample, random_state=1)
        
        # plot histogram of selected roi and frequency.
        if roi % 50 == 0 and int(freq) % 3 == 0:
            ax = sns.histplot(entropy, x=str(roi), hue="group", element="step")
            sns.move_legend(ax, "upper left")

            figure_name = 'entropy_histogram_freq_' + freq + '_roi_' + str(roi) + '.png'
            plt.savefig(output_dir + figure_name)
            plt.clf()
    
        # t-test between OCD and CN for each roi and frequency: 
        ad = entropy.query('group == "hc"')[str(roi)]
        cn = entropy.query('group == "ocd"')[str(roi)]
        ttest = stats.ttest_ind(ad, cn, permutations=10000, random_state=rng)
        result_table.loc[int(freq), (roi, 'p value')] = ttest.pvalue
        result_table.loc[int(freq), (roi, 'cohen''s d')] = cohend(ad, cn)
        
     
        # break

    ## line plot of mean values across subject of entropy for each frequency.
    ## x: roi index
    ## y: entropy
    
    ## selected groups to be displayed. add by xin May-04-2022.
    
    data_mean = data.groupby('group').mean().reset_index()
    data_mean = pd.melt(data_mean, id_vars = ['group'], value_vars = [str(i) for i in range(num_roi)])
    data_mean

    p = sns.lineplot(data=data_mean, x="variable", y="value", hue="group")
    p.set_xlabel("roi index", fontsize = 20)
    p.set_ylabel("mean entropy across subjects", fontsize = 20)

    # only show every 10 roi index.
    ax = plt.gca()
    temp = ax.xaxis.get_ticklabels()
    temp = list(set(temp) - set(temp[::10]))

    for label in temp:
        label.set_visible(False)

    figure_name = 'entropy_mean_freq_' + freq + '.png'
    plt.savefig(output_dir + figure_name)
    plt.clf()
    
    # break


ocd_out04_power_histogram_and_entropy/out04_entropy_freq_00.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_01.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_02.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_03.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_04.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_05.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_06.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_07.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_08.csv
ocd_out04_power_histogram_and_entropy/out04_entropy_freq_09.csv


<Figure size 1440x576 with 0 Axes>

In [ ]:
pd.concat([data[['ocd', 'subject_id']].groupby('group').count()], axis = 1)

In [ ]:

def add_stars(report, start_col = 0):
    
    cols = slice(0,None,2)
    report2 = report.copy()
    
    report2.iloc[:, cols] = report2.iloc[:, cols].astype(float).round(3)
    # report.iloc[:,1:]=report.iloc[:,1:].mask(report.iloc[:,1:].le(0.05), report.astype(str).apply(lambda x : x.str[:5]).add('*'))

    report2.loc[:,:] = report2.loc[:,:].astype(str).apply(lambda x : x.str[:5]).apply(lambda x : x.str.ljust(5, fillchar='0'))

    report2[report.iloc[:, cols].le(0.05)] = report2[
        report.iloc[:, cols].le(0.05)].astype(str).apply(lambda x : x.str[:5]).add('*')

    report2[report.iloc[:, cols].le(0.01)] = report2[
        report.iloc[:, cols].le(0.01)].astype(str).apply(lambda x : x.str[:5]).add('**')

    # report2[report.iloc[:,skip_col:].le(0.001)] = report2[
    #     report.iloc[:,skip_col:].le(0.001)].astype(str).apply(lambda x : x.str[:5]).add('***')
    report2[report.iloc[:, cols].le(0.001)] = '<.001***'
    
    return report2
    
report = add_stars(result_table, start_col = 0)
report.transpose().to_csv(output_dir + 'ttest_entropy_ad.csv')

report_emci = add_stars(result_table_emci, start_col = 0)
report_emci.transpose().to_csv(output_dir + 'ttest_entropy_emci.csv')

In [ ]:
report_emci.iloc[:,::2].transpose()

In [ ]:
report.iloc[:,::2].transpose()